<div align="center">
    <h1>Activation Maximization on C3D</h1>
</div>

In [4]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPool3D, ZeroPadding3D
from keras.layers.core import Dense, Dropout, Flatten

model = Sequential()

# 1st layer group
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', name='conv1', strides=(1, 1, 1), input_shape=(16, 112, 112, 3)))
model.add(MaxPool3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool1'))
# 2nd layer group
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same', name='conv2', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool2'))
# 3rd layer group
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3a', strides=(1, 1, 1)))
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool3'))
# 4th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool4'))
# 5th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5b', strides=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool5'))
model.add(Flatten())
# FC layers group
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(.5))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(.5))
model.add(Dense(487, activation='softmax', name='fc8'))

Lets load wights and compile the model

In [ ]:
model.load_weights('sports1M_weights_tf.h5')
model.compile(loss='mean_squared_error', optimizer='sgd')

Getting indices of convolutional layers

In [6]:
import keras
layer_indices = []
number_of_layers = len(model.layers)
for i in range(number_of_layers):
    if type(model.layers[i]) is keras.layers.convolutional.Conv3D:
        layer_indices.append(i)
layer_indices

[0, 2, 4, 5, 7, 8, 10, 11]

Getting number of filters for each convolution layer and total number of filters

In [ ]:
from vis.visualization import visualize_activation, get_num_filters

number_of_filters = []
for idx in layer_indices:
  number_of_filters.append(get_num_filters(model.layers[idx]))
number_of_filters

total_number_of_filters = sum(number_of_filters)

Getting all activations for each layer

In [ ]:
from vis.visualization import visualize_activation

n = len(layer_indices)
#all_filters[i][j], i -- layer index incurrent array, j -- filter index
count=0
all_filters = []
for l in range(n):
  filters = []
  for f in range(number_of_filters[l]):
    filters.append(visualize_activation(model, layer_idx=layer_indices[l], filter_indices=f))
    count += 1
    print("progress: " + str(count*100.0/total_number_of_filters) + " %")
  all_filters.append(filters)

Creating gifs of activations

In [ ]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import ImageSequenceClip

n = len(layer_indices)
for l in range(n):
  for f in range(number_of_filters[l]):
    clip = ImageSequenceClip(list(all_filters[l][f]), fps=10).resize(1.0)
    clip.write_gif(model.layer[layer_indices[l]].name+"_"+"filter"+str(f)+".gif", fps=10)
    print(model.layer[layer_indices[l]].name+"_"+"filter"+str(f)+".gif -- is created")

<div align="center">
    <h4>Keras C3D model pretrained on Sports1M dataset</h4>
</div>

<div align="center">
    <h4>conv1</h4>
</div>

<table border="5" bordercolor="blue" align="center">
		<tr>
			<td><img src="../data/layer0/layer0_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer0/layer0_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer0/layer0_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer0/layer0_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer0/layer0_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
 </table>

<div align="center">
    <h4>conv2</h4>
</div>

<table>
<tr>
			<td><img src="../data/layer2/layer2_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer2/layer2_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer2/layer2_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer2/layer2_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer2/layer2_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv3a</h4>
</div>

<table>
		<tr>
			<td><img src="../data/layer4/layer4_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer4/layer4_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer4/layer4_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer4/layer4_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer4/layer4_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv3b</h4>
</div>

<table>
<tr>
			<td><img src="../data/layer5/layer5_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer5/layer5_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer5/layer5_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer5/layer5_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer5/layer5_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv4a</h4>
</div>

<table>
		<tr>
			<td><img src="../data/layer7/layer7_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer7/layer7_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer7/layer7_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer7/layer7_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer7/layer7_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv4b</h4>
</div>

<table>
    <tr>
			<td><img src="../data/layer8/layer8_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer8/layer8_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer8/layer8_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer8/layer8_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer8/layer8_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv5a</h4>
</div>

<table>
    		<tr>
			<td><img src="../data/layer10/layer10_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer10/layer10_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer10/layer10_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer10/layer10_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer10/layer10_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>

<div align="center">
    <h4>conv5b</h4>
</div>

<table>
    		<tr>
			<td><img src="../data/layer11/layer11_filter0.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter1.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter2.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter3.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter4.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter5.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter6.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter7.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer11/layer11_filter8.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter9.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter10.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter11.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter12.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter13.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter14.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter15.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer11/layer11_filter16.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter17.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter18.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter19.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter20.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter21.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter22.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter23.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
		<tr>
			<td><img src="../data/layer11/layer11_filter24.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter25.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter26.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter27.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter28.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter29.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter30.gif" alt="" border=3 height=100 width=100></img></td>
			<td><img src="../data/layer11/layer11_filter31.gif" alt="" border=3 height=100 width=100></img></td>
		</tr>
</table>